In [158]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [159]:
import platform
platform_name = platform.system()

In [160]:
CONFIG = {
    'test_path': 'test.csv',
    'train_path': 'train.csv',

    'DEVICE' : 'cuda',
    'SEED' : 42,
}
if platform_name == 'Windows':
    CONFIG['data_main'] = 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/'
    CONFIG['model_path'] = f"{number}_model.pkl"
else:
    CONFIG['data_main'] = '/kaggle/input/equity-post-HCT-survival-predictions/'
    CONFIG['model_path'] = "/kaggle/input/2.1.15_model.pkl/other/default/1/2.1.15_model.pkl"

In [161]:

import pandas as pd
import joblib 

In [162]:
data_exp = pd.read_csv(f"{CONFIG['data_main']}data_dictionary.csv")

In [163]:
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")
train[['sex_1', 'sex_2']] = train['sex_match'].str.split('-', expand=True)
train.drop(columns=['ID'], inplace=True)

num_columns = []
for i, row in data_exp.iterrows():
    if row['type'] == 'Numerical':
        num_columns.append(row['variable'])
num_columns.append('efs')
cat_columns = [col for col in train.columns if col not in num_columns]
num_columns.remove('efs')
num_columns.remove('efs_time')

for col in train[cat_columns].columns:
    train[col] = train[col].astype(str)
    j_ch = ',[]{}:"\\<'  # набор символов для удаления
    for ch in j_ch:
        train[col] = train[col].apply(lambda x: str(x).replace(ch, ''))
train_one_hot = pd.get_dummies(train[cat_columns])
cat_columns = list(train_one_hot.columns)
train_one_hot = pd.concat([train_one_hot, train[num_columns]], axis=1)

In [164]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")
test[['sex_1', 'sex_2']] = test['sex_match'].str.split('-', expand=True)
num_columns = []
for i, row in data_exp.iterrows():
    if row['type'] == 'Numerical':
        num_columns.append(row['variable'])
num_columns.append('efs')
cat_columns = [col for col in test.columns if col not in num_columns]
num_columns.remove('efs')
num_columns.remove('efs_time')

for col in test[cat_columns].columns:
    test[col] = test[col].astype(str)
    j_ch = ',[]{}:"\\<' 
    for ch in j_ch:
        test[col] = test[col].apply(lambda x: str(x).replace(ch, ''))
test_one_hot = pd.get_dummies(test[cat_columns])
test_one_hot = pd.concat([test_one_hot, test[num_columns]], axis=1)

missing_columns = set(train_one_hot.columns) - set(test_one_hot.columns)
for col in missing_columns:
    test_one_hot[col] = False

C:\Users\Николай\AppData\Local\Temp\ipykernel_13868\1963333612.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_13868\1963333612.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_13868\1963333612.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [165]:
test_one_hot = test_one_hot[train_one_hot.columns]

In [166]:
final_cat = joblib.load(CONFIG['model_path'])

In [167]:
preds_test = final_cat.predict(test_one_hot)

In [168]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)

In [169]:
y_pred

,ID,prediction
0,28800,0.444504
1,28801,0.612916
2,28802,0.388857
